In [42]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [43]:
spark=SparkSession.builder.appName("csvex").getOrCreate()

/home/lplab/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [46]:
cspath="/home/lplab/Desktop/210962122/y.csv"

In [47]:
df=spark.read.csv(cspath,header=True,inferSchema=True)

In [48]:
df.show()

+-------+------+
|Integer|Square|
+-------+------+
|      1|     1|
|      2|     4|
|      3|     9|
|      4|    16|
|      5|    25|
+-------+------+



In [49]:
from pyspark.sql import functions as f
from pyspark.sql.functions import col
fdf=df.filter(col("Square")>=15)

In [50]:
fdf.show()

+-------+------+
|Integer|Square|
+-------+------+
|      4|    16|
|      5|    25|
+-------+------+



In [51]:
wdf=df.withColumn("Square",col("Square").cast("Float"))
cdf=df.withColumn("Square",col("Square")*10)
bdf=df.withColumn("cSquare",col("Square")*-1)


In [52]:
wdf.show()

+-------+------+
|Integer|Square|
+-------+------+
|      1|   1.0|
|      2|   4.0|
|      3|   9.0|
|      4|  16.0|
|      5|  25.0|
+-------+------+



In [53]:
cdf.show()

+-------+------+
|Integer|Square|
+-------+------+
|      1|    10|
|      2|    40|
|      3|    90|
|      4|   160|
|      5|   250|
+-------+------+



In [54]:
bdf.show()

+-------+------+-------+
|Integer|Square|cSquare|
+-------+------+-------+
|      1|     1|     -1|
|      2|     4|     -4|
|      3|     9|     -9|
|      4|    16|    -16|
|      5|    25|    -25|
+-------+------+-------+



In [55]:
from pyspark.sql.functions import count

fdf.select(count(fdf.Square)).show()

+-------------+
|count(Square)|
+-------------+
|            2|
+-------------+



In [56]:
fdf.show()

+-------+------+
|Integer|Square|
+-------+------+
|      4|    16|
|      5|    25|
+-------+------+



In [57]:
from pyspark.sql.functions import avg,sum
x=df.agg(avg("Square").alias("AVG"),sum("Square").alias("Sq"))

In [58]:
x.show()

+----+---+
| AVG| Sq|
+----+---+
|11.0| 55|
+----+---+



In [60]:
data = [("TWO", 2),
        ("ONE", 1),
        ("THREE", 3)]
schema = ["NAME", "INTEGER"]

df = spark.createDataFrame(data, schema=schema)

print("Original DataFrame:")
df.show()

output_path = "/home/lplab/Desktop/210962122"

df_single_partition = df.repartition(1)
df_single_partition.write.csv(output_path, header=True, mode="overwrite")



Original DataFrame:
+-----+-------+
| NAME|INTEGER|
+-----+-------+
|  TWO|      2|
|  ONE|      1|
|THREE|      3|
+-----+-------+



In [62]:
from pyspark.sql import SparkSession
from operator import add
input_path="/home/lplab/Desktop/210962122/part-00000-68922a37-9f13-48ff-a8fe-6d948c8dcd6f-c000.csv"
lines = spark.sparkContext.textFile(input_path)

word_counts = lines.flatMap(lambda line: line.split(" ")) \
                   .map(lambda word: (word, 1)) \
                   .reduceByKey(add)

print("Word Count:")
for word, count in word_counts.collect():
    print(f"{word}: {count}")

Word Count:
TWO,2: 1
NAME,INTEGER: 1
ONE,1: 1
THREE,3: 1
